In [2]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = DEVICE
torch.cuda.set_device(DEVICE)

In [4]:
type(device)

torch.device

In [ ]:
assert False

# Factory

## Model

**Input argument**: tensors \
**Output**: tensors 
- AlexNet
- ResNet  
- FrameCNN
- ShallowCNN 

In [121]:
import torch 
from torch import nn
from torchsummary import summary


In [ ]:
from models.cnn import create_alexnet

In [71]:
def test_model():
    pass

# test_model()
summary(test_model(),input_size=(512,),batch_size=32,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [32, 16]           8,208
              ReLU-2                   [32, 16]               0
           Dropout-3                   [32, 16]               0
            Linear-4                    [32, 5]              85
              ReLU-5                    [32, 5]               0
           Dropout-6                    [32, 5]               0
            Linear-7                    [32, 2]              12
Total params: 8,305
Trainable params: 8,305
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.06
Forward/backward pass size (MB): 0.02
Params size (MB): 0.03
Estimated Total Size (MB): 0.11
----------------------------------------------------------------


## Loss

**Input argument**: tensors \
**Output**: loss 
- Cross Entropy Loss (EXAMPLE)
- Triplet loss  
- NT-Xent
- InfoNCE

##    Train 

**Input argument**: model, dataloader, loss \
**Output**: model 
- Supervised learning
- Autoencoder Pretraining
- Predictive PreTraining 
- Contrastive Pretraining
- Time Contrastive learning
- Contrastive Predictive Coding 

In [1]:
from poutyne import Model,Experiment
from losses import NT_Xent
from models.utils import Classifier
from torch.nn import functional as F

import torch 
from torch import nn

In [ ]:
class Supervised_Learning(object):
    """
    Args:
    encoder_builder (func): callable function of the primary encoder (torch.nn.Module)
    batch_size (int): batch size

    kwargs:
    encoder_builder2 (func): callable function of the secondary encoder (torch.nn.Module)
    temperature (float): temperature of NT-Xent
    optimizer (func): callable function of optimizer (torch.optim.Optimizer)
    supervision (bool): trained with label with Supervised Contrastive Learning (Tian 2020)
    """

    def __init__(self,model,**kwargs):
        # kwargs
        criterion = kwargs.get('criterion',nn.CrossEntropyLoss)
        optimizer = kwargs.get('optimizer',torch.optim.Adam)
        lr = kwargs.get('lr',0.001)
        # overall
        self.model = model
        self.criterion = criterion(**kwargs)
        self.optimizer = optimizer(list(self.model.parameters()), lr=lr)

    def train(self,train_loader,epochs=250,verbose=True,rtn_history=True,device=None):
        """
        Return trained model (and history if rtn_history = True)

        Args:
        train_loader (torch.utils.data.dataloader.DataLoader) - the pair dataset
        epochs (int) - epochs
        verbose (bool) - verbose
        rtn_history (bool) - return both the encoder and history
        device (torch.device) - model to be trained on

        Return
        """
        history = {'loss':[]}
        torch.optim.Optimizer
        if device:
            self.model = self.model.to(device)
            self.criterion = self.criterion.to(device)

        for i in range(epochs):
            if verbose: print(f'Epoch {i+1} ',end='')
            for items in train_loader:

                if device:
                    X,y = [i.to(device) for i in items]

                self.optimizer.zero_grad()
                y_pred = self.model(X)

                # SupConLoss
                if self.supervision:
                    loss = self.criterion(y_pred,y)

                loss.backward()
                self.optimizer.step()

                X = X.cpu()
                y_pred = y_pred.cpu()
                y = y.cpu()
                del X,y_pred,y
                if verbose: print('>',end='')

            loss = loss.tolist()
            history['loss'].append(loss)
            if verbose: print(f' loss: {loss}')

        if device:
            self.model = self.model.cpu()
            self.criterion = self.criterion.cpu()

        if rtn_history:
            return self.model,history
        else:
            return self.model

In [15]:
type(torch.utils.data.DataLoader(torch.rand(1,1)))

torch.utils.data.dataloader.DataLoader

## Validation

**Input argument**: model, dataframe, train \
**Output**: score/scores 
- leave-One-Out Validation
- Cross Validation  
- Cross-Domain Validation
- Sample Efficiency 

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from poutyne import Model,Experiment
from data.selection import Selection
from data.torchData import DataLoading

In [49]:
### Cross Validation

NUC = 'NUC1'
ROOM = 1
BATCHSIZE = 64
READTYPE = 'npy'
NUM_WORKERS = 0
OPTIMIZER = 'adam'
LOSS = 'cross_entropy'
METRICS = ['accuracy','fscore_macro']
EPOCHS = 5
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)


def data_preparation(dataframe,test_sub,transform,**kwargs):
    # parameters
    nuc = kwargs.get('nuc',NUC)
    room = kwargs.get('room',ROOM)
    batch_size = kwargs.get('batch_size',BATCHSIZE)
    readtype = kwargs.get('readtype',READTYPE)
    num_workers = kwargs.get('num_workers',NUM_WORKERS)
    # selection
    selection = Selection(split='loov',test_sub=test_sub,nuc=nuc,room=room)
    df_train,_,df_test = dataselection(dataframe)
    # loading
    data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                               num_workers=num_workers,shuffle=False)
    test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                               num_workers=num_workers,shuffle=False)
    train_loader = data_loading(df_train)
    test_loader  = test_loading(df_test)
    return train_loader,test_loader

def append_record(history,record,metrics):
    df = pd.DataFrame(history)
    row = df.loc[len(df)-1,metrics]
    record.append(row)
    return record

def leaveOneOut_crossValidation(model,dataframe,transform,verbose=True,**kwargs):
    
    device = kwargs.get('device',DEVICE)
    optimizer = kwargs.get('optimizer',OPTIMIZER)
    loss = kwargs.get('loss',LOSS)
    metrics = kwargs.get('metrics',METRICS)
    epochs = kwargs.get('epochs',EPOCHS)
    
    records = []
    
    for test_sub in dataframe['person'].unique():
        
        if verbose: print(test_sub)
        
        model.build()
        
        train_loader,test_loader = data_preparation(dataframe,test_sub,transform)
        
        mdl = Model(model,OPTIMIZER,LOSS,batch_metrics=METRICS).to(device)
        history = mdl.fit(train_loader, test_loader, epochs=EPOCHS)
        records = append_record(history,records,METRICS)
    
    average_scores = pd.DataFrame(records).mean()
    return average_scores

## Misc

# Laboratory

In [1]:
import os
import sys
from time import gmtime, strftime
import numpy as np
import pandas as pd

import sklearn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor, nn
import poutyne
from poutyne import Model,Experiment

from data.custom_data import filepath_dataframe
from data.selection import Selection
from data.transformation import Transform_CnnLstmS,Transform_CnnS
from data.torchData import DataLoadings,DataLoading

from data.custom_data import nucPaired_fpDataframe
from data.torchData import PairDataLoading,DataLoading
from training.contrastive_pretraining import Contrastive_PreTraining
from training.finetuning import FineTuneCNN
from validation.loov import leaveOneOut_crossValidation

import models

#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## I/O directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
record_outpath = './record'

# data selection
dataselection_name = 'EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced'

data_selection = Selection(split='random',test_sub=0.2,val_sub=0.1,
                           nuc='NUC1',room=1,sample_per_class=None)

# data loading
transform = Transform_CnnS()
batch_size = 64
num_workers = 0

# training
optimizer_builder = torch.optim.SGD
lr = 0.001
epochs = 1

network_name = 'AlexNet'

# Experiment Name
comment = 'TestModel'
exp_name = f'{network_name}_Supervised_{dataselection_name}_Comment-{comment}'

In [2]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())
# data preparation
df = filepath_dataframe(data_dir,splitchar)
df = nucPaired_fpDataframe(df)
df_train,df_val,df_test = data_selection(df)

Cuda Availability:  True


In [3]:
model_path = None

In [3]:
##### PRE-TRAIN ######

# data loading
pretrain_loading = PairDataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                                        num_workers=num_workers,drop_last=True,supervision=True)
pretrain_loader = pretrain_loading(df_train)

# train objective 
train_objective = Contrastive_PreTraining(models.cnn.create_alexnet,batch_size=batch_size,
                                          supervision=None,temperature=0.1/0.07)
encoder,history = train_objective.train(pretrain_loader,epochs=epochs,device=device)
# save model 
model_path = f'./models/saved_models/Encoder___{exp_name}'
torch.save(encoder.state_dict(),model_path)
del encoder

Epoch 1 >>>>>>>>>>>>>>>>>>>> loss: 6.898321151733398


In [4]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

df_train = df_train.rename(columns = {'fullpath_x':'fullpath'})
df_val   = df_val.rename(columns = {'fullpath_x':'fullpath'})
df_test  = df_test.rename(columns = {'fullpath_x':'fullpath'})

train_loader = data_loading(df_train)
val_loader   = data_loading(df_val)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=model_path,
                    encoder_builder=models.cnn.create_alexnet,
                    n_classes=df.activity.nunique())

In [ ]:
# train with poutyne
mdl = Model(model,'adam','cross_entropy',batch_metrics=['accuracy']).to(device)
history = mdl.fit_generator(train_generator=train_loader,valid_generator=test_loader,epochs=20)

In [16]:


df = df.rename(columns = {'fullpath_x':'fullpath'})
df_1 = df[df['room']==1]
average_scores = leaveOneOut_crossValidation(model,df,transform,verbose=True)

One
Epoch: 1/1 Step: 26/26 100.00% |█████████████████████████|53.40s loss: 664.328237 acc: 25.155666 fscore_micro: 0.251557 fscore_macro: 0.089718 val_loss: 1.789191 val_acc: 27.272728 val_fscore_micro: 0.272727 val_fscore_macro: 0.136806
Two
Epoch: 1/1 Step: 25/25 100.00% |█████████████████████████|59.72s loss: 318.538498 acc: 25.031766 fscore_micro: 0.250318 fscore_macro: 0.101809 val_loss: 1.793202 val_acc: 16.666668 val_fscore_micro: 0.166667 val_fscore_macro: 0.047619
Three
Epoch: 1/1 Step: 22/22 100.00% |█████████████████████████|54.87s loss: 1087.395635 acc: 48.887294 fscore_micro: 0.488873 fscore_macro: 0.232042 val_loss: 1.793351 val_acc: 16.666668 val_fscore_micro: 0.166667 val_fscore_macro: 0.047619
Four
Epoch: 1/1 Step: 27/27 100.00% |█████████████████████████|60.28s loss: 963.206195 acc: 30.930233 fscore_micro: 0.309302 fscore_macro: 0.132405 val_loss: 1.792317 val_acc: 16.666668 val_fscore_micro: 0.166667 val_fscore_macro: 0.047619
Five
Epoch: 1/1 Step: 25/25 100.00% |███

In [119]:
from torchvision.models import resnet18, vgg16, alexnet

In [134]:
k = alexnet()

In [141]:
k.classifier = Classifier(9216,16,6)

In [142]:
summary(k,(3,70,1600),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 16, 399]          23,296
              ReLU-2          [-1, 64, 16, 399]               0
         MaxPool2d-3           [-1, 64, 7, 199]               0
            Conv2d-4          [-1, 192, 7, 199]         307,392
              ReLU-5          [-1, 192, 7, 199]               0
         MaxPool2d-6           [-1, 192, 3, 99]               0
            Conv2d-7           [-1, 384, 3, 99]         663,936
              ReLU-8           [-1, 384, 3, 99]               0
            Conv2d-9           [-1, 256, 3, 99]         884,992
             ReLU-10           [-1, 256, 3, 99]               0
           Conv2d-11           [-1, 256, 3, 99]         590,080
             ReLU-12           [-1, 256, 3, 99]               0
        MaxPool2d-13           [-1, 256, 1, 49]               0
AdaptiveAvgPool2d-14            [-1, 25